# Ανίχνευση του λιπώδους ιστού με συσταδοποίηση
## Εντοπισμός της συμπεριφοράς της θερμογένεσης.
## Η καταγραφή για εννέα δείγματα αφού προηγηθεί segmentation στα πέντε ποντίκια του πειράματος
### Επιλογή εικόνων για τη μέτρηση της μέσης θερμοκρασίας και της τυπικής απόκλισης
Στο αρχείο bat_list.yml έχουμε ορίσει από ποιες εικόνες θα καταγράψουμε τη μέση θερμοκρασία ενός τετραγωνικού patch το οποίο βρίσκεται στον αυχένα ενός ποντικιού, όπως φαίνεται στην παραπάνω εικόνα. 

Δομή του αρχείου yaml: 
```
samples: [0h, 24h, 48h, 72h, 96h, 120h, 144h, 192h, 240h]
mouse_id: [mouse_1, mouse_2, mouse_3, mouse_4, mouse_5]
mouse_1:
  csv: [CSV_2060.csv, CSV_2305.csv, CSV_2509.csv, CSV_2730.csv, CSV_2957.csv, CSV_3181.csv, CSV_3397.csv, CSV_3652.csv, CSV_3855.csv]
  jpeg: [IR_2060.jpg, IR_2305.jpg, IR_2509.jpg, IR_2730.jpg, IR_2957.jpg, IR_3181.jpg, IR_3397.jpg, IR_3652.jpg, IR_3855.jpg]
.
.
. κλπ
```

In [3]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from skimage.io import imread
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
from celluloid import Camera
from IPython.display import HTML # to show the animation in Jupyter
from tqdm import tqdm

from skimage.exposure import histogram
from skimage.util import img_as_ubyte

from src.features import mouse_detection
from src.data import data_

In [5]:
# Εισαγωγή του αρχείου yaml
import yaml
from src.features import img_to_vectors
with open(module_path + '/data/bat_list.yml') as file:
    experiment = yaml.load(file, Loader=yaml.FullLoader)
# experiment['samples'], experiment['mouse_id']

In [6]:
# Πρόσβαση στα δεδομένα του αρχείου
mouse_names = ['mouse_1', 'mouse_2', 'mouse_3', 'mouse_4', 'mouse_5']
# mouse_names = ['mouse_1']
# Κατασκευή των global path για τα αρχεία του πειράματος
infrared_items = []
csv_items = []
for mouse_name in mouse_names:
    for hour, jpeg_file in zip(experiment['samples'], experiment[mouse_name]['jpeg']):
        full_jpeg_path = module_path + '/data/raw/' + hour + '/' + mouse_name + '/' + jpeg_file
        infrared_items.append(full_jpeg_path)

    for hour, csv_file in zip(experiment['samples'], experiment[mouse_name]['csv']):
        full_csv_path = module_path + '/data/raw/' + hour + '/' + mouse_name + '/' + csv_file
        csv_items.append(full_csv_path)
# Εισαγωγή των δεδομένων με βάση τα path και οι προεπεξεργασίες τους
infrared_images = [] # will store (238, 98), (229, 100) arrays etc
exif_datum = []
for item in infrared_items:
    infrared_image, exif_data = initial_data_pick.IR_fixer(fpath=item, fname='')
    infrared_images.append(infrared_image)
    exif_datum.append(exif_data)

markerBack, markerBody = 70, 120

object_images, object_masks, initial_masks, mouse_locations = mouse_detection.mouse_detection(infrared_images, markerBack, markerBody)

csv_files = [] # will store (240, 160) arrays
for item in csv_items:
    csv_files.append(initial_data_pick.CSV_fixer(item, fname='', save_dir=''))

thermals = [mask * csv for (mask, csv) in zip(initial_masks, csv_files)]
new = [thermal[mouse_location] for thermal, mouse_location in zip(thermals, mouse_locations)]

fig = plt.figure(figsize=(8, 8), constrained_layout=False)
fig.suptitle(t=f'Experiment course for {mouse_names}', fontsize='xx-large')
grid = fig.add_gridspec(nrows=1, ncols=1, wspace=0.2, hspace=0.25)
ax0 = fig.add_subplot(grid[0])

mean_roi_temp = []
roi_area_area = []
std_roi = []
camera = Camera(fig)
print(f'{mouse_names}')
for i, (image) in tqdm(enumerate(new)):
    ordinatesX, ordinatesY = img_to_vectors.img_to_vectors(image)
    intensityFeature = image[ordinatesX, ordinatesY]
    xmin = int(0.35*image.shape[1])
    xmax = int(0.65*image.shape[1])
    ymin = int(0.70*image.shape[0])
    ymax = int(0.78*image.shape[0])
    roi_area = image[ymin:ymax, xmin:xmax]
    std_roi.append(np.std(roi_area))
    mean_roi_temp.append(np.mean(roi_area))
    roi_area_area.append(roi_area.size)
    # print(f'sample: #{i}')
    print(f'Body : shape: {image.shape}, area: {intensityFeature.size} Mean Temp: {np.mean(intensityFeature)}  Std Temp: {np.std(intensityFeature)}')
    print(f'Patch : shape: {roi_area.shape}, area: {roi_area.size} Mean Temp: {np.mean(roi_area)}  Std Temp: {np.std(roi_area)}')
    ax0.imshow(X=image, cmap=plt.cm.nipy_spectral)
    # ax0.set_title(label=sample)
    ax0.axvspan(xmin=xmin, xmax=xmax, color='white', alpha=0.55)
    ax0.axhspan(ymin=ymin, ymax=ymax, color='white', alpha=0.55)    
    fig.add_subplot(ax0)
    camera.snap()
animation = camera.animate(interval=100)
# το HTML πρέπει να τρέχει στο jupyter notebook 
HTML(animation.to_html5_video()) # displaying the animation

ValueError: operands could not be broadcast together with shapes (240,60) (240,160) 